In [ ]:
import pandas as pd
import requests
import json
from datetime import datetime, timedelta
import pytz

In [ ]:
berlinTz = pytz.timezone("Europe/Berlin") 
today = datetime.now(berlinTz).date()
tomorrow= today + timedelta(days=1)
api_key = " API Key "
def get_weather_data(cities):
    weather_dic={'city':[],'date_time':[],'temperature':[],'pressure':[],
    'wind_speed':[],'visibility':[],'weather_condition':[],
    'detailed_condition':[],'clouds':[],'rain':[]}

    for city in cities:
        url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={api_key}&units=metric"
        response=requests.get(url)
        weather_des=response.json()

        for item in weather_des['list']:
            weather_dic['date_time'].append(item['dt_txt'])   #Time of data forecasted (3 hours interval)
            weather_dic['temperature'].append(item['main']['temp'])   #Temperature, celsius
            weather_dic['pressure'].append(item['main']['pressure'])  #Atmospheric pressure on the sea level unit:hpa(hectopascal=millibar)
            weather_dic['wind_speed'].append(item['wind']['speed'])   #Wind speed, m/s
            weather_dic['visibility'].append(item['visibility'])   #Average visibility, metres
            weather_dic['weather_condition'].append(item['weather'][0]['main'])   #Group of weather parameters (Rain, Snow etc.)
            weather_dic['detailed_condition'].append(item['weather'][0]['description']) #Weather condition within the group
            try:
                weather_dic['rain'].append(item['rain']['3h']) #Rain volume for last 3 hours, mm
            except:
                weather_dic['rain'].append(0)    
            weather_dic['clouds'].append(item['clouds']['all'])        #Cloudiness, %    
        weather_dic['city'].extend([weather_des['city']['name']] * len(weather_des['list']))   #city     
        weather_data=pd.DataFrame(weather_dic)
        weather_data=weather_data.loc[pd.to_datetime(weather_data['date_time']).dt.date==tomorrow].reset_index(drop=True)
    return weather_data

In [ ]:
list_of_cities=['Berlin','London','Madrid','Milan','Munich','Paris']
df=get_weather_data(list_of_cities)
df

In [ ]:
import sqlalchemy

In [ ]:
schema="db_weather_flights"   # name of the database you want to use here
host="127.0.0.1"              # to connect to your local server
user="root"
password="Your Password"     # your password!!!!
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [ ]:
df.to_sql('weather',              # 'cities'-> table name;
              if_exists='append', # if_exists -> will create new table if doesn't exist, otherwise, 'append' - will append data to existing table;
              con=con,            # con-> connection string;
              index=False) 